Import libraries

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import shap
import numpy as np
import helperfunc


Load the data

In [99]:
# Load the data
df = pd.read_csv("validation_cohort_lab_1.csv")
X = df.drop(columns=['samples'])
y = df['flu']
df

,samples,0.81_84.0447m/z,0.81_130.0507m/z,10.34_106.0865m/z,4.73_422.1307m/z,9.34_349.0774n,10.87_249.1085m/z,9.28_956.3750n,10.89_352.2131n,7.88_86.0965m/z,...,8.36_144.0935n,10.23_227.0793m/z,8.65_211.1376m/z,1.30_230.0961m/z,10.33_178.1441m/z,11.61_102.1268m/z,2.11_232.1182m/z,7.00_634.7114m/z,3.21_201.0740m/z,flu
0,set2_41,1.442443e+03,215986.85000,3.264634e+04,9.585929,12556.667180,270.217328,2087.755122,NaN,9.652781e+05,...,64333.41123,5228.513750,21.615857,NaN,5731.906391,3.033089e+05,125506.553400,33.441435,15280.318500,0
1,set2_42,4.842628e+02,383635.43420,2.144310e+04,8.298590,7735.875350,9.134454,1185.473945,NaN,1.456196e+06,...,135968.12830,3400.835294,75.265402,NaN,5533.863459,1.483598e+05,11211.675050,13.611119,14012.042540,0
2,set2_43,1.398318e+03,172146.10630,3.397794e+04,13.623832,7091.784685,1291.651630,1795.695142,19.578062,1.133122e+06,...,90140.60525,3273.860087,110.042957,NaN,6319.952865,1.847441e+05,107456.392900,38.621637,13391.244450,0
3,set2_44,1.335038e+03,294313.32080,1.762505e+04,12.514224,8745.142684,73.320673,1326.096224,15.608419,1.403639e+06,...,135181.62990,4651.298094,54.036417,NaN,8465.624644,1.387979e+05,27838.916360,29.262734,13271.859880,0
4,set2_45,1.081331e+03,129673.62560,2.263049e+04,37.594386,9151.910137,1021.955165,1726.298487,NaN,8.173155e+05,...,70414.75568,527698.034600,6987.605532,NaN,5296.223571,1.907061e+05,35670.111730,48.126276,14402.488030,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,set1_92,1.210455e+06,247697.48870,3.730282e+05,31204.340620,307.981487,62006.981000,9209.039456,14.747917,1.023573e+04,...,22091.35707,82160.690360,2982.637336,43.936547,14414.926080,2.108340e+06,73248.266600,14338.988990,18.383491,1
92,set1_93,1.470304e+06,221129.88820,7.358352e+05,21950.689880,415.065923,83359.689650,3698.160463,13.473296,5.600218e+03,...,29468.95412,184380.468000,125.900555,17.947978,13198.663120,2.540869e+06,76633.353520,15051.915690,35.675958,1
93,set1_94,1.339075e+06,199333.64700,6.689625e+05,28372.001100,83.087814,84439.500460,4843.771068,12.174423,3.961192e+03,...,23690.97841,90720.312900,171.454943,38.148740,17382.179770,2.305420e+06,29977.208210,16714.523580,35.056077,1
94,set1_95,1.619613e+06,318489.97320,1.277618e+06,18151.371420,269.323002,62954.268790,6479.077248,22.231343,4.330627e+03,...,37829.61424,36630.450960,510.764012,27.392361,16949.951160,2.756049e+06,6074.210292,16150.324550,52.352145,1


Fill Null Values

In [100]:
import pandas as pd
import numpy as np
import re

# Assuming data_raw is already defined and loaded with data
data_raw =X
# Step 1: Calculate the mean for each column and replace missing values
for column in data_raw.columns:
    if data_raw[column].isnull().any():
        mean_value = data_raw[column].mean()
        data_raw[column].fillna(mean_value, inplace=True)

# Step 2: Clean column names
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
data_raw.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in data_raw.columns.values]

# Step 3: Save the entire DataFrame to a new CSV file
# data_raw.to_csv('data_raw_full.csv', index=False)


Prepare data

In [101]:
X = data_raw.drop(columns=['flu'])
y = df['flu']

Load model and evaluate

In [89]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the models
model_2 = joblib.load('Pickel_models/top_model_2.pkl')
model_3 = joblib.load('Pickel_models/top_model_3.pkl')
model_4 = joblib.load('Pickel_models/top_model_4.pkl')

# Load the external data
# Assuming X and y are already defined as the features and target variable respectively
X_external = X
y_true = y

# Scale the external data
scaler = StandardScaler()
X_external_scaled = scaler.fit_transform(X_external)

# Get probabilities from each model
prob_2 = model_2.predict_proba(X_external_scaled)
prob_3 = model_3.predict_proba(X_external_scaled)
prob_4 = model_4.predict_proba(X_external_scaled)



In [102]:
prob1_folds1 = [np.array(prob_2), np.array(prob_2), np.array(prob_2), np.array(prob_2), np.array(prob_2)]
prob1_folds2 = [np.array(prob_3), np.array(prob_3), np.array(prob_3), np.array(prob_3), np.array(prob_3)]
prob1_folds3 = [np.array(prob_4), np.array(prob_4), np.array(prob_4), np.array(prob_4), np.array(prob_4)]

In [103]:
y_folds = [np.array(y_true), np.array(y_true), np.array(y_true), np.array(y_true), np.array(y_true)]


Evaluate the model

In [104]:
Result2, prb2,prb02,ytest2,tar2,pred2=helperfunc.stacking_classification(prob1_folds1,prob1_folds2,prob1_folds3,y_folds)

0 is [[240   0]
 [  0 240]]
1 is [[240   0]
 [ 10 230]]
2 is [[240   0]
 [  0 240]]


`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTr

3 is [[240   0]
 [  0 240]]
4 is [[235   5]
 [ 10 230]]
5 is [[240   0]
 [ 10 230]]
6 is [[240   0]
 [  0 240]]
7 is [[240   0]
 [  0 240]]
8 is [[240   0]
 [  0 240]]
9 is [[240   0]
 [  0 240]]
[18:59:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "iterations", "loss_function" } are not used.

[18:59:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "iterations", "loss_function" } are not used.

[18:59:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "iterations", "loss_function" } are not used.

[18:59:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.


Results

In [105]:
Result2

,Accuracy,Precision,Recall,Specificity,F1-score,Auc
MLPClassifier,100.00,100.00,100.00,100.00,100.00,1.000000
LinearDiscriminantAnalysis,97.92,98.00,97.92,97.92,97.92,1.000000
XGBClassifier,100.00,100.00,100.00,100.00,100.00,1.000000
RandomForestClassifier,100.00,100.00,100.00,100.00,100.00,1.000000
LogisticRegression,96.88,96.90,96.88,96.88,96.88,0.996962
SVM,97.92,98.00,97.92,97.92,97.92,1.000000
ExtraTreesClassifier,100.00,100.00,100.00,100.00,100.00,1.000000
AdaBoostClassifier,100.00,100.00,100.00,100.00,100.00,1.000000
KNeighborsClassifier,100.00,100.00,100.00,100.00,100.00,1.000000
GradientBoostingClassifier,100.00,100.00,100.00,100.00,100.00,1.000000


In [106]:
Result2.to_csv('Pickel_models/results/externalvalid.csv')